# Loading Packages

In [2]:
import re 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
import random 
import math
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as torch_utils
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn.functional as F
import d2l
import time
import traceback
import fastprogress
from torchmetrics.classification import BinaryAccuracy, Accuracy 
import torch.nn.init as init
import torch.optim.lr_scheduler as lr_scheduler
from itertools import repeat
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
seed = 100
torch.manual_seed(seed)
np.random.seed(seed)

In [4]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Set the device to the first available GPU
    device = torch.device("cuda:0")
else:
    # If GPU is not available, use the CPU
    device = torch.device("cpu")

# Model Architecture

In [5]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2, bidirectional=False):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.gru = nn.GRUCell(input_dim, hidden_dim, bias=True)
        #Xavier initialization for GRU weights
        #for name, param in self.gru.named_parameters():
        #    if 'weight' in name:
        #        init.xavier_uniform_(param.data)
        #    elif 'bias' in name:
        #        init.constant_(param.data, 0.0)
                
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sotfplus = nn.Softplus()
        self.relu = nn.ReLU()

        
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.sotfplus(out[:,-1]))
        #out = self.fc(self.relu(out[:,-1]))
        out = F.softmax(out, dim=1)
        return out, h
    
    #def init_hidden(self, batch_size):
        #weight = next(self.parameters()).data
        #hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        #return hidden
    def init_hidden(self, batch_size):
        if batch_size > 1:
            weight = next(self.parameters()).data
            hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        else:
            weight = next(self.parameters()).data
            hidden = weight.new(self.n_layers, self.hidden_dim).zero_().to(device)
        return hidden

In [6]:
class GRUCellNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.gru = nn.GRUCell(input_dim, hidden_dim, bias=True)                
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softplus = nn.Softplus()
        self.relu = nn.ReLU()

        self.gru_cells = nn.ModuleList([
            nn.GRUCell(input_dim, hidden_dim) if i == 0 else nn.GRUCell(hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        self.batch_norm = nn.BatchNorm1d(hidden_dim)  # Add BatchNorm outside GRU cells

        
        
    def forward(self, x, h=None, hidden_states=None):
        if h is None:
            h = [torch.zeros(x.size(0), self.hidden_dim) for _ in range(self.num_layers)]

        hidden_states = []  # Initialize a list to store hidden states at each time step

        for t in range(x.size(1)):
            input_t = x[:, t, :]
            new_hidden_states = []

            for layer_idx, gru_cell in enumerate(self.gru_cells):
                h[layer_idx] = gru_cell(input_t, h[layer_idx])
                new_hidden_states.append(h[layer_idx])
                input_t = h[layer_idx]

            hidden_states.append(new_hidden_states)  # Append hidden states for the current time step

        last_hidden_states = [layer_states[-1] for layer_states in hidden_states]
        last_hidden_states[-1] = self.batch_norm(last_hidden_states[-1])
        out = self.fc(self.softplus(last_hidden_states[-1]))

        # Return the output and the list of hidden states for the entire sequence
        return out, last_hidden_states, hidden_states

# Training functions

In [7]:
# Custom Dataset class for loading data
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        # Implement data retrieval for each index
        input_data = self.X[idx]
        target_data = self.y[idx]
        input_data = input_data.unsqueeze(0)
        
        # Convert data to torch tensors if required
        input_tensor = torch.Tensor(input_data)
        target_tensor = torch.Tensor(target_data)
        
        return input_tensor, target_tensor

In [8]:
def accuracy(correct, total):
    return float(correct)/total


In [9]:
def train(dataloader, model, optimizer, loss_fn, scheduler=None, device=None):
    epoch_loss = []
    epoch_correct, epoch_total = 0, 0
    model = model.to(device)
    model.train()
    predicted_probs = []
    predicted_labels = []
    hidden_states = []
    for x, y in dataloader:
        x = x.to(device).float()
        y = y.to(device).float()
        
        optimizer.zero_grad()
        hidden_states_batch = []  # Initialize a list to store hidden states for the current batch        
        out, last_hidden_states = model(x)
        y_prob = torch.sigmoid(out)
        # Append the predicted probabilities to the list
        predicted_probs.append(y_prob.cpu().detach().numpy())
        loss = loss_fn(out, y)
        epoch_loss.append(loss.item())

        #hidden_states.append(hidden)
        
        y_pred = torch.round(y_prob)
        # Append the batch's hidden states to the list
        for layer_state in last_hidden_states:
            hidden_states_batch.append(layer_state.cpu().detach().numpy())
        epoch_correct += sum((y == y_pred).flatten()).item()
        epoch_total += y.numel()
        # Append the batch's hidden states to the overall hidden_states list
        hidden_states.append(hidden_states_batch)
        
        loss.backward()
        torch_utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()

        predicted_labels.extend(zip(y_pred.cpu().detach().cpu().numpy(), y.cpu().numpy()))
        if scheduler:
            scheduler.step(loss)
    
    return np.mean(epoch_loss), accuracy(epoch_correct, epoch_total), predicted_labels, predicted_probs 

In [10]:
def validate(dataloader, model, loss_fn, device=None):
    epoch_loss = []
    epoch_correct, epoch_total = 0, 0
    model = model.to(device).float()
    model.eval()
    predicted_probs = []
    predicted_labels = []
    #hidden_states = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device).float()
            y = y.to(device).float()
            
            out, last_hidden_states = model(x)
            
            loss = loss_fn(out, y)
            epoch_loss.append(loss.item())

            #hidden_states.append(hidden)
            y_pred = torch.sigmoid(out)
            predicted_probs.append(y_pred.cpu().detach().numpy())
            y_pred = torch.round(y_pred)
            epoch_correct += sum((y == y_pred).flatten())
            epoch_total += y.numel()
            predicted_labels.extend(zip(y_pred.cpu().numpy(), y.cpu().numpy()))
    
    return np.mean(epoch_loss), accuracy(epoch_correct, epoch_total), predicted_labels, predicted_probs

In [11]:
def run_training(train_dataloader, val_dataloader, model, optimizer, loss_fn, num_epochs, scheduler=None, device=None, schedule_on_train=True, verbose=True):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    #train_hidden_states, val_hidden_states = [], []
    
    for epoch in range(num_epochs):
        epoch_train_loss, epoch_train_acc, train_preds, train_probs = train(train_dataloader, model, optimizer, loss_fn, scheduler, device)
        
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)

        #train_hidden_states.extend(train_hidden)
        
        if val_dataloader is not None:
            epoch_val_loss, epoch_val_acc, val_preds, val_probs = validate(val_dataloader, model, loss_fn, device)
        
            val_losses.append(epoch_val_loss)
            val_accs.append(epoch_val_acc)

            #val_hidden_states.extend(val_hidden)
        
        #if isinstance(scheduler, ReduceLROnPlateau):
        #    scheduler.step(epoch_train_acc if schedule_on_train or val_dataloader is None else epoch_val_acc)
            
        if epoch % 10 == 0:
            val_str = f", val loss: {epoch_val_loss}, val acc: {epoch_val_acc}" if val_dataloader is not None else ""
            print(f"Epoch {epoch}, train loss: {epoch_train_loss}, train acc: {epoch_train_acc}{val_str}")
        if epoch == num_epochs - 1:  # Store values only for the final epoch
            train_predicted_labels = train_preds
            #val_predicted_labels = val_preds
            train_probs_final = train_probs
            #val_probs_final = val_probs
            if val_dataloader is not None:
                val_predicted_labels = val_preds
                val_probs_final = val_probs

    if val_dataloader is not None:        
        return train_losses, train_accs, val_losses, val_accs, train_predicted_labels, val_predicted_labels, train_probs_final, val_probs_final
    else: 
        return train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final

# training loop 

In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = ['s_1','s_2','s_3','s_4']
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = 4
hidden_dim = 4
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=10, verbose=True, threshold=0.01, threshold_mode='rel')

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(10,101,step=10):
        shift = i
        set_values = j
        model_name = 's_'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial', 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 's_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")

 # Iterate through the test data batches
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 's_results_train.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")

In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 34
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(10,101,step=10):
        shift = i
        set_values = j
        model_name = 's_m'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'sm_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")

 # Iterate through the test data batches
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'sm_results_train.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    








In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']

df_human = pd.read_csv("data_human_jarvis.csv")
df_human = df_human.drop(columns=['H_id','s_1','s_2','s_3','s_4'])
file = "data_model_v3.csv"
df = pd.read_csv(file)
# List of column names to drop

df = df.drop(columns=columns_to_drop)
# Concatenate the two dataframes vertically (along rows)
df = pd.concat([df_human,df], axis=1)

# Reset the index of the concatenated dataframe
df.reset_index(drop=True, inplace=True)

labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial','H_trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
scaled_df = scaled_df.dropna(axis=0)
cols_to_omit = ['id','trial']
# Use the drop method to exclude columns
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 46
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(10,101,step=10):
        shift = i
        set_values = j
        model_name = 's_m_h'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
      
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4','H_headFront_x_minus_'+str(shift), 'H_headFront_y_minus_'+str(shift), 
        'H_headFront_z_minus_'+str(shift), 'H_neck_x_minus_'+str(shift),'H_neck_y_minus_'+str(shift), 'H_neck_z_minus_'+str(shift), 'H_lowerBack_x_minus_'+str(shift), 'H_lowerBack_y_minus_'+str(shift),'H_lowerBack_z_minus_'+str(shift),
        'H_leftWrist_x_minus_'+str(shift), 'H_leftWrist_y_minus_'+str(shift), 'H_leftWrist_z_minus_'+str(shift),'H_leftShoulder_x_minus_'+str(shift), 'H_leftShoulder_y_minus_'+str(shift), 'H_leftShoulder_z_minus_'+str(shift),
       'H_leftElbow_x_minus_'+str(shift), 'H_leftElbow_y_minus_'+str(shift), 'H_leftElbow_z_minus_'+str(shift)]
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'smh_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
            
            
 # Iterate through the test data batches
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'smh_results_train.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    








In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift
shifted_df = pd.DataFrame()

        # Loop through unique trial values
for trial_value in scaled_df['trial'].unique():
        # Filter the DataFrame for the current trial
        trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

        # Create shifted columns for each column in columns_to_shift
        for col in columns_to_shift:
            new_col_name = col + '_minus_' + str(shift)
            trial_df[new_col_name] = trial_df[col].shift(shift)

        # Drop the last 'i' records for each trial
        trial_df = trial_df.dropna()

        # Append the modified trial_df to the shifted_df
        shifted_df = shifted_df.append(trial_df, ignore_index=True)
    
#selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
shifted_df = shifted_df[selected_columns]
# Step 5: Split the data into training and test sets based on the 'trial' column
train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
val_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
full_set = shifted_df.drop(columns=['id','trial'])

# split data into x and y 
X_train, y_train = train_set.drop(columns=labels), train_set[labels]
X_val, y_val = val_set.drop(columns=labels), val_set[labels]
X, y = full_set.drop(columns=labels), full_set[labels]

# reset index 
X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
X_val, y_val = X_val.reset_index(drop=True), y_val.reset_index(drop=True)
X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

# Create custom datasets for training, validation, and testing
full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
val_dataset = MyDataset(torch.tensor(X_val.values), torch.tensor(y_val.values))

fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=X_val.shape[0], shuffle=False)

In [ ]:

df = pd.read_csv("data_human_jarvis.csv")
# List of column names to drop
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['H_trial','H_id','s_1','s_2','s_3','s_4'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['H_trial','H_id','s_1','s_2','s_3','s_4']] = df[['H_trial','H_id','s_1','s_2','s_3','s_4']]
scaled_df = scaled_df.dropna(axis=0)
cols_to_omit = ['H_trial','H_id']
# Use the drop method to exclude columns
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
scaled_df.shape[1]-2

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 11
output_dim = 4
num_layers = 1
n_epochs =151
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(0,152,step=10):
        shift = i
        set_values = j
        model_name = 's_h'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['H_trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['H_trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['H_id', 'H_trial', 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4','H_headFront_x_minus_'+str(shift), 'H_headFront_y_minus_'+str(shift), 
        'H_headFront_z_minus_'+str(shift), 'H_neck_x_minus_'+str(shift),'H_neck_y_minus_'+str(shift), 'H_neck_z_minus_'+str(shift), 'H_lowerBack_x_minus_'+str(shift), 'H_lowerBack_y_minus_'+str(shift),'H_lowerBack_z_minus_'+str(shift),
        'H_leftWrist_x_minus_'+str(shift), 'H_leftWrist_y_minus_'+str(shift), 'H_leftWrist_z_minus_'+str(shift),'H_leftShoulder_x_minus_'+str(shift), 'H_leftShoulder_y_minus_'+str(shift), 'H_leftShoulder_z_minus_'+str(shift),
       'H_leftElbow_x_minus_'+str(shift), 'H_leftElbow_y_minus_'+str(shift), 'H_leftElbow_z_minus_'+str(shift)]
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['H_trial']!=set_values].drop(columns=['H_id', 'H_trial'])
        test_set = shifted_df[shifted_df['H_trial']==set_values].drop(columns=['H_id', 'H_trial'])
        full_set = shifted_df.drop(columns=['H_id', 'H_trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'sh_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    








# Shuffling

In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = ['s_1','s_2','s_3','s_4']
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = 4
hidden_dim = 4
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=10, verbose=True, threshold=0.01, threshold_mode='rel')

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(10,101,step=10):
        shift = i
        set_values = j
        model_name = 's_'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial', 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 
        
        #shuffling 
        y_train = y_train.sample(frac=1, random_state=10).reset_index(drop=True)
        y_test = y_test.sample(frac=1, random_state=10).reset_index(drop=True)
        y = y.sample(frac=1, random_state=10).reset_index(drop=True)

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models_shuffled' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 's_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/shuffled/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")

 # Iterate through the test data batches
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 's_results_train.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/shuffled/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")

In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 34
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(10,101,step=10):
        shift = i
        set_values = j
        model_name = 's_m'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 
        
        #shuffling 
        y_train = y_train.sample(frac=1, random_state=10).reset_index(drop=True)
        y_test = y_test.sample(frac=1, random_state=10).reset_index(drop=True)
        y = y.sample(frac=1, random_state=10).reset_index(drop=True)

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models_shuffled' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'sm_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/shuffled/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")

 # Iterate through the test data batches
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'sm_results_train.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/shuffled/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    








In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']

df_human = pd.read_csv("data_human_jarvis.csv")
df_human = df_human.drop(columns=['H_id','s_1','s_2','s_3','s_4'])
file = "data_model_v3.csv"
df = pd.read_csv(file)
# List of column names to drop

df = df.drop(columns=columns_to_drop)
# Concatenate the two dataframes vertically (along rows)
df = pd.concat([df_human,df], axis=1)

# Reset the index of the concatenated dataframe
df.reset_index(drop=True, inplace=True)

labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial','H_trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
scaled_df = scaled_df.dropna(axis=0)
cols_to_omit = ['id','trial']
# Use the drop method to exclude columns
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 46
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
# finished smh, left set 14,15 to train 
for j in np.arange(1,16):
    for i in np.arange(10,101,step=10):
        shift = i
        set_values = j
        model_name = 's_m_h'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
      
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4','H_headFront_x_minus_'+str(shift), 'H_headFront_y_minus_'+str(shift), 
        'H_headFront_z_minus_'+str(shift), 'H_neck_x_minus_'+str(shift),'H_neck_y_minus_'+str(shift), 'H_neck_z_minus_'+str(shift), 'H_lowerBack_x_minus_'+str(shift), 'H_lowerBack_y_minus_'+str(shift),'H_lowerBack_z_minus_'+str(shift),
        'H_leftWrist_x_minus_'+str(shift), 'H_leftWrist_y_minus_'+str(shift), 'H_leftWrist_z_minus_'+str(shift),'H_leftShoulder_x_minus_'+str(shift), 'H_leftShoulder_y_minus_'+str(shift), 'H_leftShoulder_z_minus_'+str(shift),
       'H_leftElbow_x_minus_'+str(shift), 'H_leftElbow_y_minus_'+str(shift), 'H_leftElbow_z_minus_'+str(shift)]
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 
        
        #shuffling 
        y_train = y_train.sample(frac=1, random_state=10).reset_index(drop=True)
        y_test = y_test.sample(frac=1, random_state=10).reset_index(drop=True)
        y = y.sample(frac=1, random_state=10).reset_index(drop=True)

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)


        train_losses, train_accs, val_losses, val_accs, train_predicted_labels, train_probs_final = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)



        state_dict = model.state_dict()

        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models_shuffled' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)


        
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_test.columns, index=y_test.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_test).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'smh_results.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/shuffled/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
            
            
 # Iterate through the test data batches
        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        test_accs = np.mean(accuracies)
        
        
        file_name = 'smh_results_train.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/test/shuffled/" + file_name  # Replace with your desired file path
        value_to_save = model_name+": "+str(test_accs)  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    








# Overfitting check

In [ ]:

df = pd.read_csv("data_human_jarvis.csv")
# List of column names to drop
labels = ['s_1','s_2','s_3','s_4']
df.rename(columns={'H_id': 'id', 'H_trial': 'trial'}, inplace=True)
data_to_scale = df.drop(columns=['trial','id','s_1','s_2','s_3','s_4'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['trial','id','s_1','s_2','s_3','s_4']] = df[['trial','id','s_1','s_2','s_3','s_4']]
scaled_df = scaled_df.dropna(axis=0)
cols_to_omit = ['trial','id']
# Use the drop method to exclude columns
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 11
output_dim = 4
num_layers = 1
n_epochs =151
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(0,151,step=10):
        shift = i
        set_values = j
        model_name = 's_h'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial', 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4','H_headFront_x_minus_'+str(shift), 'H_headFront_y_minus_'+str(shift), 
        'H_headFront_z_minus_'+str(shift), 'H_neck_x_minus_'+str(shift),'H_neck_y_minus_'+str(shift), 'H_neck_z_minus_'+str(shift), 'H_lowerBack_x_minus_'+str(shift), 'H_lowerBack_y_minus_'+str(shift),'H_lowerBack_z_minus_'+str(shift),
        'H_leftWrist_x_minus_'+str(shift), 'H_leftWrist_y_minus_'+str(shift), 'H_leftWrist_z_minus_'+str(shift),'H_leftShoulder_x_minus_'+str(shift), 'H_leftShoulder_y_minus_'+str(shift), 'H_leftShoulder_z_minus_'+str(shift),
       'H_leftElbow_x_minus_'+str(shift), 'H_leftElbow_y_minus_'+str(shift), 'H_leftElbow_z_minus_'+str(shift)]
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id','trial'])
        val_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])
        
        

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_val, y_val = val_set.drop(columns=labels), val_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_val, y_val = X_val.reset_index(drop=True), y_val.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True)
        

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        val_dataset = MyDataset(torch.tensor(X_val.values), torch.tensor(y_val.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        val_dataloader = DataLoader(val_dataset, batch_size=X_val.shape[0], shuffle=False)
        
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models/' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        model.load_state_dict(torch.load(full_model))

        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
         # Iterate through the test data batches
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns



        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds.numpy(), columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        file_name = 'sh_overfitting_check.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/results_sm/" + file_name  # Replace with your desired file path
        value_to_save = model_name+"_"+"oc"+": "+str(np.mean(accuracies))  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
# First, group your DataFrame by the 'trial' column
trial_groups = df.groupby('trial')

# Get a list of trial group names (trial IDs)
trial_ids = list(trial_groups.groups.keys())

# Shuffle the trial IDs randomly
random.shuffle(trial_ids)

# Create an empty list to store shuffled trial DataFrames
shuffled_trial_dfs = []

# Iterate through the shuffled trial IDs and add the corresponding trial DataFrames to the list
for trial_id in trial_ids:
    shuffled_trial_dfs.append(trial_groups.get_group(trial_id))

# Concatenate the shuffled trial DataFrames back into a single DataFrame
shuffled_df = pd.concat(shuffled_trial_dfs, ignore_index=True)

In [ ]:

shift = 2
set_values = 5
model_name = 's_h'+str(shift)+'_'+'val'+str(set_values)


# Create an empty DataFrame to store the shifted data
shifted_df = pd.DataFrame()

# Loop through unique trial values
for trial_value in scaled_df['trial'].unique():
    # Filter the DataFrame for the current trial
    trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

    # Create shifted columns for each column in columns_to_shift
    for col in columns_to_shift:
        new_col_name = col + '_minus_' + str(shift)
        trial_df[new_col_name] = trial_df[col].shift(shift)

    # Drop the last 'i' records for each trial
    trial_df = trial_df.dropna()

    # Append the modified trial_df to the shifted_df
    shifted_df = shifted_df.append(trial_df, ignore_index=True)

#selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
selected_columns = ['id', 'trial', 's_1_minus_'+str(shift),
's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4','H_headFront_x_minus_'+str(shift), 'H_headFront_y_minus_'+str(shift), 
'H_headFront_z_minus_'+str(shift), 'H_neck_x_minus_'+str(shift),'H_neck_y_minus_'+str(shift), 'H_neck_z_minus_'+str(shift), 'H_lowerBack_x_minus_'+str(shift), 'H_lowerBack_y_minus_'+str(shift),'H_lowerBack_z_minus_'+str(shift),
'H_leftWrist_x_minus_'+str(shift), 'H_leftWrist_y_minus_'+str(shift), 'H_leftWrist_z_minus_'+str(shift),'H_leftShoulder_x_minus_'+str(shift), 'H_leftShoulder_y_minus_'+str(shift), 'H_leftShoulder_z_minus_'+str(shift),
'H_leftElbow_x_minus_'+str(shift), 'H_leftElbow_y_minus_'+str(shift), 'H_leftElbow_z_minus_'+str(shift)]
shifted_df = shifted_df[selected_columns]
# Step 5: Split the data into training and test sets based on the 'trial' column
train_set = shifted_df[shifted_df['trial']!=set_values]
val_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
full_set = shifted_df.drop(columns=['id','trial'])



# Assuming you have a DataFrame 'train_set' with a 'trial' column
# and you want to shuffle the trials while preserving the order of samples within each trial.

# First, group your DataFrame by the 'trial' column
trial_groups = train_set.groupby('trial')

# Get a list of trial group names (trial IDs)
trial_ids = list(trial_groups.groups.keys())

# Shuffle the trial IDs randomly
random.shuffle(trial_ids)

# Create an empty list to store shuffled trial DataFrames
shuffled_trial_dfs = []

# Iterate through the shuffled trial IDs and add the corresponding trial DataFrames to the list
for trial_id in trial_ids:
    shuffled_trial_dfs.append(trial_groups.get_group(trial_id))

# Concatenate the shuffled trial DataFrames back into a single DataFrame
train_set = pd.concat(shuffled_trial_dfs, ignore_index=True)
print(train_set)


In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 34
output_dim = 4
num_layers = 1
n_epochs =151
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(1,142,step=10):
        shift = i
        set_values = j
        model_name = 's_m'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values]
        val_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])        

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_val, y_val = val_set.drop(columns=labels), val_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
                        # First, group your DataFrame by the 'trial' column
        trial_groups = X_train.groupby('trial')

        # Get a list of trial group names (trial IDs)
        trial_ids = list(trial_groups.groups.keys())

        # Shuffle the trial IDs randomly
        random.shuffle(trial_ids)

        # Create an empty list to store shuffled trial DataFrames
        shuffled_trial_dfs = []

        # Iterate through the shuffled trial IDs and add the corresponding trial DataFrames to the list
        for trial_id in trial_ids:
            shuffled_trial_dfs.append(trial_groups.get_group(trial_id))
        
        X_train = X_train.drop(columns=['id','trial'])
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_val, y_val = X_val.reset_index(drop=True), y_val.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True)
        

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        val_dataset = MyDataset(torch.tensor(X_val.values), torch.tensor(y_val.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        val_dataloader = DataLoader(val_dataset, batch_size=X_val.shape[0], shuffle=False)
        
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models/' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        model.load_state_dict(torch.load(full_model))

        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
         # Iterate through the test data batches
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns



        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds.numpy(), columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        file_name = 'sm_shuffled.txt'
        file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/results_sm/" + file_name  # Replace with your desired file path
        value_to_save = model_name+"_"+"oc"+": "+str(np.mean(accuracies))  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
df_human = pd.read_csv("data_human_jarvis.csv")
df_human = df_human.drop(columns=['H_id','s_1','s_2','s_3','s_4'])
file = "data_model_v3.csv"
df = pd.read_csv(file)
# List of column names to drop

df = df.drop(columns=columns_to_drop)
# Concatenate the two dataframes vertically (along rows)
df = pd.concat([df_human,df], axis=1)

# Reset the index of the concatenated dataframe
df.reset_index(drop=True, inplace=True)

labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial','H_trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
scaled_df = scaled_df.dropna(axis=0)
cols_to_omit = ['id','trial']
# Use the drop method to exclude columns
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
len(columns_to_shift)

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 46
output_dim = 4
num_layers = 1
n_epochs =201
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
for j in np.arange(1,16):
    for i in np.arange(0,151,step=10):
        shift = i
        set_values = j
        model_name = 's_m_h'+str(shift)+'_'+'val'+str(set_values)


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4','H_headFront_x_minus_'+str(shift), 'H_headFront_y_minus_'+str(shift), 
        'H_headFront_z_minus_'+str(shift), 'H_neck_x_minus_'+str(shift),'H_neck_y_minus_'+str(shift), 'H_neck_z_minus_'+str(shift), 'H_lowerBack_x_minus_'+str(shift), 'H_lowerBack_y_minus_'+str(shift),'H_lowerBack_z_minus_'+str(shift),
        'H_leftWrist_x_minus_'+str(shift), 'H_leftWrist_y_minus_'+str(shift), 'H_leftWrist_z_minus_'+str(shift),'H_leftShoulder_x_minus_'+str(shift), 'H_leftShoulder_y_minus_'+str(shift), 'H_leftShoulder_z_minus_'+str(shift),
       'H_leftElbow_x_minus_'+str(shift), 'H_leftElbow_y_minus_'+str(shift), 'H_leftElbow_z_minus_'+str(shift)]
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values]
        val_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])
        
        

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_val, y_val = val_set.drop(columns=labels), val_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
                # First, group your DataFrame by the 'trial' column
        trial_groups = X_train.groupby('trial')

        # Get a list of trial group names (trial IDs)
        trial_ids = list(trial_groups.groups.keys())

        # Shuffle the trial IDs randomly
        random.shuffle(trial_ids)

        # Create an empty list to store shuffled trial DataFrames
        shuffled_trial_dfs = []

        # Iterate through the shuffled trial IDs and add the corresponding trial DataFrames to the list
        for trial_id in trial_ids:
            shuffled_trial_dfs.append(trial_groups.get_group(trial_id))

        # Concatenate the shuffled trial DataFrames back into a single DataFrame
        X_train = pd.concat(shuffled_trial_dfs, ignore_index=True)
        X_train = train_set.drop(columns=['id', 'trial'])
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_val, y_val = X_val.reset_index(drop=True), y_val.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True)
        X_train = X_train.drop(columns=labels)
        
        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        val_dataset = MyDataset(torch.tensor(X_val.values), torch.tensor(y_val.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        val_dataloader = DataLoader(val_dataset, batch_size=X_val.shape[0], shuffle=False)
        
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models/' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        model.load_state_dict(torch.load(full_model))

        model.eval()  # Set the model to evaluation mode
        # Initialize an empty list to store predictions
        all_preds = []
        all_probs = []
         # Iterate through the test data batches
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, _ = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)

        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns



        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds.numpy(), columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        accuracies = (predictions_df == y_train).mean()

        print("Accuracy for each output state:")
        print(np.mean(accuracies))
        #file_name = 'smh_shuffled.txt'
        #file_path = "C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/results_sm/" + file_name  # Replace with your desired file path
        value_to_save = model_name+"_"+"oc"+": "+str(np.mean(accuracies))  # Replace with the value you want to save

        try:
            with open(file_path, "a") as file:
                # 2. Write the value to the file
                file.write(value_to_save)
                print(f"Value '{value_to_save}' saved to '{file_path}' successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
df = pd.read_csv('models_comparison_v3.csv')

# Set the "shift" column as the index (optional, for better x-axis labels)
df = df[::-1]
df.set_index('T', inplace=True)

# Create a line plot for each model's performance
#plt.figure(figsize=(10, 6))

# Define custom colors for each model
colors = {
    'null': 'silver',
    'solo': 'magenta',
    'dyadic': 'blue'
}

# Create a line plot for each model's performance with custom colors
plt.figure(figsize=(7, 5))

for model in ['null','solo','dyadic']:
    plt.plot(df.index, df[model], marker='o', label=model, color=colors[model])

# Set plot labels and title
plt.xlim(max(df.index),min(df.index))
plt.xlabel('T (shift in seconds)')
plt.ylabel('Accuracy')
#plt.title('Performance of Four Models')

# Add a legend
plt.legend()

# Show the plot
plt.grid(False)
plt.show()

In [ ]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
df_human = pd.read_csv("data_human_jarvis.csv")
df_human = df_human.drop(columns=['H_id','s_1','s_2','s_3','s_4'])
file = "data_model_v3.csv"
df = pd.read_csv(file)
# List of column names to drop

df = df.drop(columns=columns_to_drop)
# Concatenate the two dataframes vertically (along rows)
df = pd.concat([df_human,df], axis=1)

# Reset the index of the concatenated dataframe
df.reset_index(drop=True, inplace=True)

labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial','H_trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
scaled_df = scaled_df.dropna(axis=0)
cols_to_omit = ['id','trial']
# Use the drop method to exclude columns
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

In [ ]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 46
output_dim = 4
num_layers = 1
n_epochs =201
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

# Heatmaps

In [ ]:
# Define the labels for the y-axis (predicted states)
y_labels = ['1','2','3','4']

# Create a figure and axis
fig, ax = plt.subplots()

# Create the heatmap plot
heatmap = ax.imshow(all_probs_array.T, cmap='coolwarm', aspect='auto', interpolation='none')

# Add colorbar
cbar = plt.colorbar(heatmap)

# Set the axis labels and title
ax.set_xlabel('Time Step')
ax.set_ylabel('Predicted State')
ax.set_title('Probabilities of Predicting Output States')

# Set the y-axis ticks and labels
ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels(y_labels)

# Show the plot

In [ ]:
# Define the labels for the y-axis (predicted states)
y_labels = ['1', '2', '3', '4']

# Create a figure with a grid of subplots (2 rows, 1 column)
fig, axs = plt.subplots(2, 1, figsize=(8, 10))

# Create the heatmap plot in the first subplot
heatmap = axs[0].imshow(all_probs_array.T, cmap='coolwarm', aspect='auto', interpolation='none')

# Add colorbar to the heatmap subplot
cbar = plt.colorbar(heatmap, ax=axs[0])

# Set the axis labels and title for the heatmap subplot
axs[0].set_xlabel('Time Step')
axs[0].set_ylabel('Predicted State')
axs[0].set_title('Probabilities of Predicting Output States')

# Set the y-axis ticks and labels for the heatmap subplot
axs[0].set_yticks(np.arange(len(y_labels)))
axs[0].set_yticklabels(y_labels)

# Create the second plot (line plot) in the second subplot
axs[1].plot(np.arange(len(y_val)), y_val, color='green', label='Actual Values')
axs[1].set_xlabel('Time Step')
axs[1].set_ylabel('Actual Value')
axs[1].set_title('Actual Test Values for Y')
axs[1].legend()

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
actual_values = y_val
# Define the labels for the y-axis (predicted states)
y_labels = ['1', '2', '3', '4']

# Create a figure with a grid of subplots (2 rows, 1 column)
fig, axs = plt.subplots(2,1, figsize=(8,10))

# Create the first heatmap plot in the first subplot
heatmap1 = axs[0].imshow(all_probs_array.T, cmap='coolwarm', aspect='auto', interpolation='none')

# Add colorbar to the first heatmap subplot
cbar1 = plt.colorbar(heatmap1, ax=axs[0])

# Set the axis labels and title for the first heatmap subplot
#axs[0].set_xlabel('Time Step')
#axs[0].set_ylabel('Predicted State')
axs[0].set_title('Predicted States Probabilities')

# Set the y-axis ticks and labels for the first heatmap subplot
axs[0].set_yticks(np.arange(len(y_labels)))
axs[0].set_yticklabels(y_labels)

# Create the second heatmap plot in the second subplot
heatmap2 = axs[1].imshow(actual_values.T, cmap='coolwarm', aspect='auto', interpolation='none')

# Add colorbar to the second heatmap subplot
cbar2 = plt.colorbar(heatmap2, ax=axs[1])

# Set the axis labels and title for the second heatmap subplot
axs[1].set_xlabel('Time Step')
#axs[1].set_ylabel('Predicted State')
axs[1].set_title('True States')

# Set the y-axis ticks and labels for the second heatmap subplot
axs[1].set_yticks(np.arange(len(y_labels)))
axs[1].set_yticklabels(y_labels)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

# gates and hidden states

In [144]:
class GRUCellNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.gru = nn.GRUCell(input_dim, hidden_dim, bias=True)                
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softplus = nn.Softplus()
        self.relu = nn.ReLU()

        self.gru_cells = nn.ModuleList([
            nn.GRUCell(input_dim, hidden_dim) if i == 0 else nn.GRUCell(hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        self.batch_norm = nn.BatchNorm1d(hidden_dim)  # Add BatchNorm outside GRU cells

        
        
    def forward(self, x, h=None, hidden_states=None):
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_dim)  # Initialize hidden state for the single layer

        hidden_states = []  # Initialize a list to store hidden states at each time step

        for t in range(x.size(1)):
            input_t = x[:, t, :]
            h = self.gru(input_t, h)  # Update the hidden state for the single layer
            hidden_states.append(h)  # Append the hidden state for the current time step

        h = self.batch_norm(h)  # Apply batch normalization to the final hidden state
        out = self.fc(self.softplus(h))  # Calculate the output based on the final hidden state
        out = torch.sigmoid(out)
        out = torch.round(out)

        # Return the output and the list of hidden states for the entire sequence
        return out, h, hidden_states

In [16]:
def train(dataloader, model, optimizer, loss_fn, scheduler=None, device=None):
    epoch_loss = []
    epoch_correct, epoch_total = 0, 0
    model = model.to(device)
    model.train()
    predicted_probs = []
    predicted_labels = []
    hidden_states = []  # Initialize a list to store hidden states

    for x, y in dataloader:
        x = x.to(device).float()
        y = y.to(device).float()
        
        optimizer.zero_grad()
        
        out, last_hidden_states, hidden_states_batch = model(x)  # Collect hidden states here
        y_prob = torch.sigmoid(out)
        # Append the predicted probabilities to the list
        predicted_probs.append(y_prob.cpu().detach().numpy())
        loss = loss_fn(out, y)
        epoch_loss.append(loss.item())
        
        hidden_states.append(hidden_states_batch)  # Append hidden states to the list
        
        y_pred = torch.round(y_prob)
        epoch_correct += sum((y == y_pred).flatten()).item()
        epoch_total += y.numel()
        
        loss.backward()
        torch_utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()

        predicted_labels.extend(zip(y_pred.cpu().detach().cpu().numpy(), y.cpu().numpy()))
        if scheduler:
            scheduler.step()
    
    return np.mean(epoch_loss), accuracy(epoch_correct, epoch_total), predicted_labels, predicted_probs, hidden_states

In [17]:
def validate(dataloader, model, loss_fn, device=None):
    epoch_loss = []
    epoch_correct, epoch_total = 0, 0
    model = model.to(device).float()
    model.eval()
    predicted_probs = []
    predicted_labels = []
    hidden_states = []  # Initialize a list to store hidden states

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device).float()
            y = y.to(device).float()
            
            out, last_hidden_states, hidden_states_batch = model(x)  # Collect hidden states here
            
            loss = loss_fn(out, y)
            epoch_loss.append(loss.item())
            
            hidden_states.append(hidden_states_batch)  # Append hidden states to the list
            
            y_pred = torch.sigmoid(out)
            predicted_probs.append(y_pred.cpu().detach().numpy())
            y_pred = torch.round(y_pred)
            epoch_correct += sum((y == y_pred).flatten())
            epoch_total += y.numel()
            predicted_labels.extend(zip(y_pred.cpu().numpy(), y.cpu().numpy()))
    
    return np.mean(epoch_loss), accuracy(epoch_correct, epoch_total), predicted_labels, predicted_probs, hidden_states

In [18]:
def run_training(train_dataloader, val_dataloader, model, optimizer, loss_fn, num_epochs, scheduler=None, device=None, schedule_on_train=True, verbose=True):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    train_hidden_states, val_hidden_states = [], [] # Initialize lists to store hidden states
    
    for epoch in range(num_epochs):
        epoch_train_loss, epoch_train_acc, train_preds, train_probs, train_hidden = train(train_dataloader, model, optimizer, loss_fn, scheduler, device)
        
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)

        train_hidden_states.extend(train_hidden)  # Append hidden states to the list
        
        if val_dataloader is not None:
            epoch_val_loss, epoch_val_acc, val_preds, val_probs, val_hidden = validate(val_dataloader, model, loss_fn, device)
        
            val_losses.append(epoch_val_loss)
            val_accs.append(epoch_val_acc)

            val_hidden_states.extend(val_hidden)  # Append hidden states to the list
        
        #if isinstance(scheduler, ReduceLROnPlateau):
        #    scheduler.step(epoch_train_acc if schedule_on_train or val_dataloader is None else epoch_val_acc)
            
        if epoch % 50 == 0:
            val_str = f", val loss: {epoch_val_loss}, val acc: {epoch_val_acc}" if val_dataloader is not None else ""
            print(f"Epoch {epoch}, train loss: {epoch_train_loss}, train acc: {epoch_train_acc}{val_str}")
        if epoch == num_epochs - 1:  # Store values only for the final epoch
            train_predicted_labels = train_preds
            #val_predicted_labels = val_preds
            train_probs_final = train_probs
            #val_probs_final = val_probs
            if val_dataloader is not None:
                val_predicted_labels = val_preds
                val_probs_final = val_probs

    if val_dataloader is not None:        
        return train_losses, train_accs, val_losses, val_accs, train_predicted_labels, val_predicted_labels, train_probs_final, val_probs_final, train_hidden_states, val_hidden_states
    else: 
        return train_losses, train_accs, train_predicted_labels, train_probs_final, train_hidden_states

In [232]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns
columns_to_shift

Index(['nose_x', 'nose_y', 'nose_z', 'headTop_x', 'headTop_y', 'headTop_z',
       'neck_x', 'neck_y', 'neck_z', 'tailBase_x', 'tailBase_y', 'tailBase_z',
       'lEar_x', 'lEar_y', 'lEar_z', 'lShoulder_x', 'lShoulder_y',
       'lShoulder_z', 'lElbow_x', 'lElbow_y', 'lElbow_z', 'lWrist_x',
       'lWrist_y', 'lWrist_z', 'lHip_x', 'lHip_y', 'lHip_z', 'rEar_x',
       'rEar_y', 'rEar_z', 'rShoulder_x', 'rShoulder_y', 'rShoulder_z',
       'rElbow_x', 'rElbow_y', 'rElbow_z', 'rWrist_x', 'rWrist_y', 'rWrist_z',
       'rHip_x', 'rHip_y', 'rHip_z', 's_1', 's_2', 's_3', 's_4'],
      dtype='object')

In [ ]:
for x, y in train_dataloader:
        x = x.to(device).float()
        y = y.to(device).float()
        
        optimizer.zero_grad()
        
        out, last_hidden_states, batch_hidden_states = model(x)

In [ ]:
batch_hidden_states[0][0]

In [ ]:
print(batch_hidden_states[0][0].shape)

In [ ]:
hidden_states_array = batch_hidden_states[0][0].detach().numpy()
hidden_states_array = hidden_states_array.T 

In [ ]:
hidden_states_array.shape

In [ ]:
# Create a figure with a single subplot
fig, axs = plt.subplots(1, 1, figsize=(14,6))  # Adjust the figsize as needed

# Create the heatmap plot in the single subplot
heatmap = axs.imshow(hidden_states_array, cmap='coolwarm', aspect='auto', interpolation='none')

# Add colorbar to the heatmap subplot
cbar = plt.colorbar(heatmap, ax=axs)

# Set the axis labels and title for the heatmap subplot
axs.set_ylabel('hidden unit')
axs.set_xlabel('time step')
axs.set_title('Heatmap of hidden states')

# Adjust spacing between the subplot
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
scaled_df.shape[1]-2

In [233]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 34
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [234]:
shift_values = np.array([10,80,130])

In [235]:
#hidden_states_all = []
hidden_states_dict = {}
for j in np.arange(1,16):
    for i in shift_values:
        shift = i
        set_values = j
        model_name = 's_m'+str(shift)+'_'+'val'+str(set_values)+'_gates'


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)
        #print(X_train.shape[0])
        

        train_losses, train_accs, train_predicted_labels, train_probs_final, train_hidden_states = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)
        
        #hidden_states_dict[(shift, set_values)] = train_hidden_states
        
        all_preds = []
        all_probs = []
        hidden_states = []

        # Iterate through the test data batches
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, h, test_hidden_states = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)
            hidden_states.append(test_hidden_states)
        
        #print(hidden_states[0][0])    
        hidden_states_array = hidden_states[0][0].detach().numpy()
        hidden_states_array = hidden_states_array.T
        #print(hidden_states_array.shape) 
        
        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()


        columns = ['s_1','s_2','s_3','s_4']
        all_probs_df = pd.DataFrame(all_probs_array)
        all_probs_df.columns = columns
        # Convert the tensor of predictions to a DataFrame
        predictions_df = pd.DataFrame(all_preds_array, columns=y_train.columns, index=y_train.index)
        # Calculate accuracy for each output state
        
        # Create a figure with a single subplot
        fig, axs = plt.subplots(2, 1, figsize=(16,10))  # Adjust the figsize as needed

        # Create the heatmap plot in the single subplot
        heatmap = axs[0].imshow(hidden_states_array, cmap='seismic', aspect='auto', interpolation='none')

        # Add colorbar to the heatmap subplot
        #cbar = plt.colorbar(heatmap, ax=axs)

        # Set the axis labels and title for the heatmap subplot
        axs[0].set_ylabel('hidden unit')
        axs[0].set_xlabel('time step')
        axs[0].set_title('Heatmap of hidden states for s='+str(i)+' trial='+str(j))
        
        axs[1].imshow(all_probs_array.T, cmap='seismic', aspect='auto', interpolation='none')
        y_labels = ['1', '2', '3', '4']
        axs[1].set_yticks(np.arange(len(y_labels)))
        axs[1].set_yticklabels(y_labels)
        
        # Add colorbar to the heatmap subplot
        #cbar = plt.colorbar(heatmap, ax=axs)
        plot_title = 'solo_s'+str(i)+'_val'+str(j)
        #plt.title(plot_title)
        # Set the axis labels and title for the heatmap subplot
        axs[0].set_ylabel('predictions')
        axs[0].set_xlabel('time step')
        #axs[0].set_title()
        # Adjust spacing between the subplot
        plt.tight_layout()
        # Show the plot
        # Specify the file path
        # Specify the directory where you want to save the plots
        save_directory = r'C:\Users\kacpe\Desktop\study\research lab\lab_rotation_git\hidden_states_heatmaps_train'

        # Create the directory if it doesn't exist
        os.makedirs(save_directory, exist_ok=True)

        # Define the plot title
        plot_title = f'solo_s{i}_val{j}'

        # Specify the file path using os.path.join
        file_path = os.path.join(save_directory, f'{plot_title}.png')

        # Save the plot to the specified path
        fig.savefig(file_path)
        #plt.show()
        plt.close()
                
        
       
    
    

Epoch 0, train loss: 0.99318927526474, train acc: 0.6004935459377373
Epoch 50, train loss: 0.08497070521116257, train acc: 0.9844343204252088
Epoch 0, train loss: 0.43113425374031067, train acc: 0.8743532560214095
Epoch 50, train loss: 0.18211303651332855, train acc: 0.9378679750223016
Epoch 0, train loss: 0.3486967086791992, train acc: 0.8874617737003058
Epoch 50, train loss: 0.2525496780872345, train acc: 0.904638124362895
Epoch 0, train loss: 0.43490543961524963, train acc: 0.9182550115018074
Epoch 50, train loss: 0.07446524500846863, train acc: 0.9850065724613868
Epoch 0, train loss: 0.3527289927005768, train acc: 0.8772032902467685
Epoch 50, train loss: 0.20687879621982574, train acc: 0.9199471210340776
Epoch 0, train loss: 0.3726874589920044, train acc: 0.8681343622333182
Epoch 50, train loss: 0.34488749504089355, train acc: 0.8746028143440763
Epoch 0, train loss: 0.20412753522396088, train acc: 0.9521866582158136
Epoch 50, train loss: 0.16878153383731842, train acc: 0.9604262398

In [27]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 34
output_dim = 4
num_layers = 1
n_epochs =100
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [243]:
#hidden_states_all = []
hidden_states_dict = {}
for j in np.arange(13,15):
    for i in shift_values:
        shift = i
        set_values = j
        model_name = 's_m'+str(shift)+'_'+'val'+str(set_values)+'_gates'


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])

        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 

        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)
        #print(X_train.shape[0])
        

        train_losses, train_accs, train_predicted_labels, train_probs_final, train_hidden_states = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)
        
        #hidden_states_dict[(shift, set_values)] = train_hidden_states
        
        all_preds = []
        all_probs = []
        hidden_states = []

        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, h, test_hidden_states = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

            # Append predictions to the list
            all_preds.append(preds)
            all_probs.append(probabilities)
            hidden_states.append(test_hidden_states)
        
        #print(hidden_states[0][0])    
        hidden_states_array = hidden_states[0][0].detach().numpy()
        hidden_states_array = hidden_states_array.T
        #print(hidden_states_array.shape) 
        
        # Concatenate the predicted batches
        all_preds = torch.cat(all_preds, dim=0)
        all_probs = torch.cat(all_probs, dim=0)

        all_preds_array = all_preds.numpy()
        all_probs_array = all_probs.numpy()
        h_name = 'hidden_s'+str(i)+'_trial'+str(j)
        p_name = 'probs_s'+str(i)+'_trial'+str(j)
        np.save(h_name, hidden_states_array)
        np.save(p_name, all_probs_array)



                
        
       
    
    

Epoch 0, train loss: 0.14604417979717255, train acc: 0.9692057704502219
Epoch 50, train loss: 0.14604417979717255, train acc: 0.9692057704502219
Epoch 0, train loss: 0.19220440089702606, train acc: 0.9305086336336337
Epoch 50, train loss: 0.19220440089702606, train acc: 0.9305086336336337
Epoch 0, train loss: 0.32724639773368835, train acc: 0.8729472774416595
Epoch 50, train loss: 0.32724639773368835, train acc: 0.8729472774416595
Epoch 0, train loss: 0.14533913135528564, train acc: 0.9692511225144324
Epoch 50, train loss: 0.14533913135528564, train acc: 0.9692511225144324
Epoch 0, train loss: 0.19234581291675568, train acc: 0.9297945205479452
Epoch 50, train loss: 0.19234581291675568, train acc: 0.9297945205479452
Epoch 0, train loss: 0.3151167333126068, train acc: 0.8735733099209834
Epoch 50, train loss: 0.3151167333126068, train acc: 0.8735733099209834


In [240]:
np.save('hidden_states.npy', hidden_states_array)

In [ ]:
first_value = list(hidden_states_dict.values())[0]
type(first_value)
hidden_states_array = first_value[0][0].detach().numpy()
hidden_states_array = hidden_states_array.T 

In [ ]:
# Create a figure with a single subplot
fig, axs = plt.subplots(2, 1, figsize=(14,6))  # Adjust the figsize as needed

# Create the heatmap plot in the single subplot
heatmap = axs[0].imshow(hidden_states_array, cmap='seismic', aspect='auto', interpolation='none')

# Add colorbar to the heatmap subplot
cbar = plt.colorbar(heatmap, ax=axs)

# Set the axis labels and title for the heatmap subplot
axs[0].set_ylabel('hidden unit')
axs[0].set_xlabel('time step')
axs[0].set_title('Heatmap of hidden states')



# Adjust spacing between the subplot
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
        # Specify the folder path and the model filename
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/models' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)

# gradients

In [24]:
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
file = "data_model_v3.csv"
df = pd.read_csv(file)
df = df.drop(columns=columns_to_drop)
labels = ['s_1','s_2','s_3','s_4']
data_to_scale = df.drop(columns=['id', 'trial'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)
scaled_df[['id', 'trial']] = df[['id', 'trial']]
cols_to_omit = ['id','trial']
columns_to_shift = scaled_df.drop(cols_to_omit, axis=1).columns

columns_to_shift

Index(['nose_x', 'nose_y', 'nose_z', 'headTop_x', 'headTop_y', 'headTop_z',
       'neck_x', 'neck_y', 'neck_z', 'tailBase_x', 'tailBase_y', 'tailBase_z',
       'lEar_x', 'lEar_y', 'lEar_z', 'lShoulder_x', 'lShoulder_y',
       'lShoulder_z', 'lElbow_x', 'lElbow_y', 'lElbow_z', 'lWrist_x',
       'lWrist_y', 'lWrist_z', 'lHip_x', 'lHip_y', 'lHip_z', 'rEar_x',
       'rEar_y', 'rEar_z', 'rShoulder_x', 'rShoulder_y', 'rShoulder_z',
       'rElbow_x', 'rElbow_y', 'rElbow_z', 'rWrist_x', 'rWrist_y', 'rWrist_z',
       'rHip_x', 'rHip_y', 'rHip_z', 's_1', 's_2', 's_3', 's_4'],
      dtype='object')

In [25]:
# Hyperparameters
input_dim = scaled_df.shape[1]-2
hidden_dim = 34
output_dim = 4
num_layers = 1
n_epochs =151
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [43]:
np.arange(13,14)

array([13])

In [103]:
#hidden_states_all = []
hidden_states_dict = {}
for j in np.arange(13,14):
    for i in np.arange(80,81):
        shift = i
        set_values = j
        model_name = 's_m'+str(shift)+'_'+'val'+str(set_values)+'_gates'


        # Create an empty DataFrame to store the shifted data
        shifted_df = pd.DataFrame()

        # Loop through unique trial values
        for trial_value in scaled_df['trial'].unique():
            # Filter the DataFrame for the current trial
            trial_df = scaled_df[scaled_df['trial'] == trial_value].copy()

            # Create shifted columns for each column in columns_to_shift
            for col in columns_to_shift:
                new_col_name = col + '_minus_' + str(shift)
                trial_df[new_col_name] = trial_df[col].shift(shift)

            # Drop the last 'i' records for each trial
            trial_df = trial_df.dropna()

            # Append the modified trial_df to the shifted_df
            shifted_df = shifted_df.append(trial_df, ignore_index=True)
        
        #selected_columns = ['id', 'trial','s_1_minus_'+str(shift),'s_2_minus_'+str(shift),'s_3_minus_'+str(shift),'s_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        selected_columns = ['id', 'trial','nose_x_minus_'+str(shift), 'nose_y_minus_'+str(shift),
        'nose_z_minus_'+str(shift), 'headTop_x_minus_'+str(shift), 'headTop_y_minus_'+str(shift),
        'headTop_z_minus_'+str(shift), 'neck_x_minus_'+str(shift), 'neck_y_minus_'+str(shift),
        'neck_z_minus_'+str(shift), 'tailBase_x_minus_'+str(shift), 'tailBase_y_minus_'+str(shift),
        'tailBase_z_minus_'+str(shift), 'lEar_x_minus_'+str(shift), 'lEar_y_minus_'+str(shift),
        'lEar_z_minus_'+str(shift), 'lShoulder_x_minus_'+str(shift), 'lShoulder_y_minus_'+str(shift),
        'lShoulder_z_minus_'+str(shift), 'lElbow_x_minus_'+str(shift), 'lElbow_y_minus_'+str(shift),
        'lElbow_z_minus_'+str(shift), 'lWrist_x_minus_'+str(shift), 'lWrist_y_minus_'+str(shift),
        'lWrist_z_minus_'+str(shift), 'lHip_x_minus_'+str(shift), 'lHip_y_minus_'+str(shift),
        'lHip_z_minus_'+str(shift), 'rEar_x_minus_'+str(shift), 'rEar_y_minus_'+str(shift), 'rEar_z_minus_'+str(shift),
        'rShoulder_x_minus_'+str(shift), 'rShoulder_y_minus_'+str(shift), 'rShoulder_z_minus_'+str(shift),
        'rElbow_x_minus_'+str(shift), 'rElbow_y_minus_'+str(shift), 'rElbow_z_minus_'+str(shift),
        'rWrist_x_minus_'+str(shift), 'rWrist_y_minus_'+str(shift), 'rWrist_z_minus_'+str(shift),
        'rHip_x_minus_'+str(shift), 'rHip_y_minus_'+str(shift), 'rHip_z_minus_'+str(shift), 's_1_minus_'+str(shift),
        's_2_minus_'+str(shift), 's_3_minus_'+str(shift), 's_4_minus_'+str(shift),'s_1','s_2','s_3','s_4']
        shifted_df = shifted_df[selected_columns]
        # Step 5: Split the data into training and test sets based on the 'trial' column
        train_set = shifted_df[shifted_df['trial']!=set_values].drop(columns=['id', 'trial'])
        test_set = shifted_df[shifted_df['trial']==set_values].drop(columns=['id', 'trial'])
        full_set = shifted_df.drop(columns=['id','trial'])
        
        # split data into x and y 
        X_train, y_train = train_set.drop(columns=labels), train_set[labels]
        X_test, y_test = test_set.drop(columns=labels), test_set[labels]
        X, y = full_set.drop(columns=labels), full_set[labels]
        print(X_train.columns)
        # reset index 
        X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
        X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
        X, y = X.reset_index(drop=True), y.reset_index(drop=True) 
        print(X_train.columns)
        # Create custom datasets for training, validation, and testing
        full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
        train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
        test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

        fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
        train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)
        #print(X_train.shape[0])
        

        train_losses, train_accs, train_predicted_labels, train_probs_final, train_hidden_states = run_training(
            train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)
        
        
        
                # Specify the folder path and the model filename
        state_dict = model.state_dict()
        folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models' 
        model_filename = model_name + '.pth'  

        # Combine the folder path and model filename
        full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
        torch.save(state_dict, full_model)
        
        # Define the folder and model filename
        load_folder = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models'
        model_filename = 's_m80_val13_gates.pth'
        
        # Iterate through the test data batches
        for inputs, _ in train_dataloader:
            inputs = inputs.float()
            inputs.requires_grad = True  # Set requires_grad to True
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, h, test_hidden_states = model(inputs)
                probabilities = torch.sigmoid(predictions)
                output = torch.round(probabilities)

# Load the model
#checkpoint = torch.load(os.path.join(load_folder, model_filename))
#model.load_state_dict(checkpoint)

# Set the model in evaluation mode
#model.eval()


#

Index(['nose_x_minus_80', 'nose_y_minus_80', 'nose_z_minus_80',
       'headTop_x_minus_80', 'headTop_y_minus_80', 'headTop_z_minus_80',
       'neck_x_minus_80', 'neck_y_minus_80', 'neck_z_minus_80',
       'tailBase_x_minus_80', 'tailBase_y_minus_80', 'tailBase_z_minus_80',
       'lEar_x_minus_80', 'lEar_y_minus_80', 'lEar_z_minus_80',
       'lShoulder_x_minus_80', 'lShoulder_y_minus_80', 'lShoulder_z_minus_80',
       'lElbow_x_minus_80', 'lElbow_y_minus_80', 'lElbow_z_minus_80',
       'lWrist_x_minus_80', 'lWrist_y_minus_80', 'lWrist_z_minus_80',
       'lHip_x_minus_80', 'lHip_y_minus_80', 'lHip_z_minus_80',
       'rEar_x_minus_80', 'rEar_y_minus_80', 'rEar_z_minus_80',
       'rShoulder_x_minus_80', 'rShoulder_y_minus_80', 'rShoulder_z_minus_80',
       'rElbow_x_minus_80', 'rElbow_y_minus_80', 'rElbow_z_minus_80',
       'rWrist_x_minus_80', 'rWrist_y_minus_80', 'rWrist_z_minus_80',
       'rHip_x_minus_80', 'rHip_y_minus_80', 'rHip_z_minus_80', 's_1_minus_80',
       's_2_m

In [130]:
# Define the folder and model filename
load_folder = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models'
model_filename = 's_m80_val13_gates.pth'

# Iterate through the test data batches
for inputs, _ in train_dataloader:
    inputs = inputs.float()
    inputs.requires_grad = True  # Set requires_grad to True
    # Forward pass to get predictions
    with torch.no_grad():
        predictions, h, test_hidden_states = model(inputs)
        probabilities = torch.sigmoid(predictions)
        output = torch.round(probabilities)
inputs.grad

In [112]:
output

tensor([[1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        ...,
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.]])

In [116]:
model(input)[0]

tensor([[15.5881, 17.0496,  7.1234, -2.7169],
        [15.6907, 16.8004,  7.1833, -2.5166],
        [14.9813, 15.7438,  6.8294, -2.4401],
        ...,
        [ 5.0684,  3.3052, -4.5890, -2.3956],
        [ 5.0625,  3.1659, -4.4379, -2.1808],
        [ 5.0584,  3.2695, -4.3667, -2.2210]], grad_fn=<AddmmBackward0>)

In [105]:

input = torch.tensor(X_train.values)
input = input.unsqueeze(1)
input = input.float()
input.shape 

torch.Size([5328, 1, 46])

In [107]:
output

(tensor([[15.5881, 17.0496,  7.1234, -2.7169],
         [15.6907, 16.8004,  7.1833, -2.5166],
         [14.9813, 15.7438,  6.8294, -2.4401],
         ...,
         [ 5.0684,  3.3052, -4.5890, -2.3956],
         [ 5.0625,  3.1659, -4.4379, -2.1808],
         [ 5.0584,  3.2695, -4.3667, -2.2210]], grad_fn=<AddmmBackward0>),
 tensor([[-0.1361, -3.9054,  4.3457,  ...,  6.9570, -0.9351, -1.0151],
         [-0.1834, -3.3678,  4.3394,  ...,  6.9214, -0.8772, -1.2564],
         [-0.1403, -3.1291,  3.9980,  ...,  6.4979, -0.8859, -1.4158],
         ...,
         [-0.8940,  0.8611, -0.3193,  ...,  0.0296,  2.5547,  2.2299],
         [-0.8732,  0.9012, -0.3054,  ...,  0.0481,  2.5995,  1.8107],
         [-0.8328,  0.9142, -0.3030,  ...,  0.0511,  2.5933,  1.5676]],
        grad_fn=<NativeBatchNormBackward0>),
 [tensor([[ 0.0378,  0.0049,  0.0404,  ...,  0.1942,  0.0496, -0.3348],
          [ 0.0359,  0.0124,  0.0403,  ...,  0.1932,  0.0511, -0.3366],
          [ 0.0376,  0.0157,  0.0368,  ...,  0

In [109]:
output.size

AttributeError: 'tuple' object has no attribute 'size'

In [98]:
model(input)

(tensor([[15.6528, 16.5354,  6.8842, -2.5370],
         [15.7095, 16.3226,  6.9530, -2.3735],
         [15.0120, 15.2652,  6.5979, -2.3344],
         ...,
         [ 4.9547,  3.1134, -4.5596, -2.0263],
         [ 4.9473,  2.9779, -4.4284, -1.8249],
         [ 4.9355,  3.0609, -4.3583, -1.8576]], grad_fn=<AddmmBackward0>),
 tensor([[-0.2498, -4.0778,  4.4233,  ...,  6.9959, -0.7348, -1.3482],
         [-0.2950, -3.5366,  4.4125,  ...,  6.9596, -0.6890, -1.5728],
         [-0.2455, -3.2876,  4.0599,  ...,  6.5503, -0.7003, -1.6831],
         ...,
         [-0.8525,  0.8381, -0.3202,  ..., -0.0396,  2.6897,  2.1940],
         [-0.8352,  0.8757, -0.3068,  ..., -0.0204,  2.7345,  1.8132],
         [-0.7982,  0.8865, -0.3056,  ..., -0.0164,  2.7295,  1.5923]],
        grad_fn=<NativeBatchNormBackward0>),
 [tensor([[ 0.0308,  0.0021,  0.0413,  ...,  0.2176,  0.0700, -0.3320],
          [ 0.0290,  0.0097,  0.0412,  ...,  0.2165,  0.0713, -0.3338],
          [ 0.0310,  0.0132,  0.0376,  ...,  0

In [119]:
output

(tensor([[15.5881, 17.0496,  7.1234, -2.7169],
         [15.6907, 16.8004,  7.1833, -2.5166],
         [14.9813, 15.7438,  6.8294, -2.4401],
         ...,
         [ 5.0684,  3.3052, -4.5890, -2.3956],
         [ 5.0625,  3.1659, -4.4379, -2.1808],
         [ 5.0584,  3.2695, -4.3667, -2.2210]], grad_fn=<AddmmBackward0>),
 tensor([[-0.1361, -3.9054,  4.3457,  ...,  6.9570, -0.9351, -1.0151],
         [-0.1834, -3.3678,  4.3394,  ...,  6.9214, -0.8772, -1.2564],
         [-0.1403, -3.1291,  3.9980,  ...,  6.4979, -0.8859, -1.4158],
         ...,
         [-0.8940,  0.8611, -0.3193,  ...,  0.0296,  2.5547,  2.2299],
         [-0.8732,  0.9012, -0.3054,  ...,  0.0481,  2.5995,  1.8107],
         [-0.8328,  0.9142, -0.3030,  ...,  0.0511,  2.5933,  1.5676]],
        grad_fn=<NativeBatchNormBackward0>),
 [tensor([[ 0.0378,  0.0049,  0.0404,  ...,  0.1942,  0.0496, -0.3348],
          [ 0.0359,  0.0124,  0.0403,  ...,  0.1932,  0.0511, -0.3366],
          [ 0.0376,  0.0157,  0.0368,  ...,  0

In [131]:
model(input)

(tensor([[15.5881, 17.0496,  7.1234, -2.7169],
         [15.6907, 16.8004,  7.1833, -2.5166],
         [14.9813, 15.7438,  6.8294, -2.4401],
         ...,
         [ 5.0684,  3.3052, -4.5890, -2.3956],
         [ 5.0625,  3.1659, -4.4379, -2.1808],
         [ 5.0584,  3.2695, -4.3667, -2.2210]], grad_fn=<AddmmBackward0>),
 tensor([[-0.1361, -3.9054,  4.3457,  ...,  6.9570, -0.9351, -1.0151],
         [-0.1834, -3.3678,  4.3394,  ...,  6.9214, -0.8772, -1.2564],
         [-0.1403, -3.1291,  3.9980,  ...,  6.4979, -0.8859, -1.4158],
         ...,
         [-0.8940,  0.8611, -0.3193,  ...,  0.0296,  2.5547,  2.2299],
         [-0.8732,  0.9012, -0.3054,  ...,  0.0481,  2.5995,  1.8107],
         [-0.8328,  0.9142, -0.3030,  ...,  0.0511,  2.5933,  1.5676]],
        grad_fn=<NativeBatchNormBackward0>),
 [tensor([[ 0.0378,  0.0049,  0.0404,  ...,  0.1942,  0.0496, -0.3348],
          [ 0.0359,  0.0124,  0.0403,  ...,  0.1932,  0.0511, -0.3366],
          [ 0.0376,  0.0157,  0.0368,  ...,  0

In [136]:
predictions, hidden, probs = model(input)
probabilities = torch.sigmoid(predictions)
output = torch.round(probabilities)

# Assuming `input` is your input data tensor
input = input.requires_grad_(True)
output.requires_grad_(True)
target.requires_grad_(True)
loss = loss_fn(output, target)
loss.backward()

# Access the gradients for the input data
input_gradients = input.grad

In [138]:
input.grad.shape

torch.Size([5328, 1, 46])

In [143]:
input.grad.squeeze(1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [66]:
        # Define the folder and model filename
load_folder = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models'
model_filename = 's_m80_val13_gates.pth'

# Load the model
checkpoint = torch.load(os.path.join(load_folder, model_filename))
model.load_state_dict(checkpoint)

# Set the model in evaluation mode
model.eval()
# Set the model in evaluation mode


# Access the model's state dictionary
model_state_dict = model.state_dict()

# Now you can access the weights of the nn.GRUCell layers
gru_cell_weights = {
    key: value
    for key, value in model_state_dict.items()
    if key.startswith('gru_cells.')
}
print(len(gru_cell_weights))
# Print or access the weights as needed
for key, value in gru_cell_weights.items():
    print(key)
    print(value)
    


# Extract and assign the weights to separate variables
weight_ih = gru_cell_weights['gru_cells.0.weight_ih']
weight_hh = gru_cell_weights['gru_cells.0.weight_hh']


4
gru_cells.0.weight_ih
tensor([[ 0.1454, -0.0767,  0.1165,  ...,  0.0882,  0.0758,  0.0415],
        [ 0.0745, -0.1682,  0.0754,  ...,  0.0346, -0.0484, -0.1178],
        [-0.0083, -0.1615,  0.1327,  ...,  0.0147, -0.1474, -0.1203],
        ...,
        [ 0.0235, -0.0237, -0.0839,  ...,  0.0383, -0.1634,  0.1645],
        [-0.0403,  0.0512,  0.0946,  ..., -0.0186,  0.1682,  0.1391],
        [ 0.1168, -0.0078, -0.1271,  ..., -0.0558,  0.1122,  0.0342]])
gru_cells.0.weight_hh
tensor([[ 0.0654,  0.0306,  0.1648,  ..., -0.0643,  0.1707,  0.0210],
        [-0.0169,  0.1101,  0.0372,  ...,  0.0406, -0.0684,  0.0486],
        [ 0.1226, -0.1662, -0.0386,  ...,  0.0828,  0.0141, -0.0264],
        ...,
        [ 0.1211, -0.1242,  0.1029,  ...,  0.0835,  0.0161,  0.0562],
        [-0.0618, -0.0938, -0.0629,  ...,  0.0225,  0.1529, -0.1611],
        [-0.0711,  0.0437, -0.0786,  ...,  0.0553,  0.0523,  0.1267]])
gru_cells.0.bias_ih
tensor([-0.0448, -0.1460, -0.1346,  0.0724, -0.1092, -0.0197,  0.0

In [67]:
len(weight_ih[0])

46

In [45]:
weight_ih_np = weight_ih.numpy()

In [58]:
features = ['nose_x_minus_80', 'nose_y_minus_80', 'nose_z_minus_80',
       'headTop_x_minus_80', 'headTop_y_minus_80', 'headTop_z_minus_80',
       'neck_x_minus_80', 'neck_y_minus_80', 'neck_z_minus_80',
       'tailBase_x_minus_80', 'tailBase_y_minus_80', 'tailBase_z_minus_80',
       'lEar_x_minus_80', 'lEar_y_minus_80', 'lEar_z_minus_80',
       'lShoulder_x_minus_80', 'lShoulder_y_minus_80', 'lShoulder_z_minus_80',
       'lElbow_x_minus_80', 'lElbow_y_minus_80', 'lElbow_z_minus_80',
       'lWrist_x_minus_80', 'lWrist_y_minus_80', 'lWrist_z_minus_80',
       'lHip_x_minus_80', 'lHip_y_minus_80', 'lHip_z_minus_80',
       'rEar_x_minus_80', 'rEar_y_minus_80', 'rEar_z_minus_80',
       'rShoulder_x_minus_80', 'rShoulder_y_minus_80', 'rShoulder_z_minus_80',
       'rElbow_x_minus_80', 'rElbow_y_minus_80', 'rElbow_z_minus_80',
       'rWrist_x_minus_80', 'rWrist_y_minus_80', 'rWrist_z_minus_80',
       'rHip_x_minus_80', 'rHip_y_minus_80', 'rHip_z_minus_80', 's_1_minus_80',
       's_2_minus_80', 's_3_minus_80', 's_4_minus_80']

In [52]:
weight_ih_np.shape

(102, 46)

In [53]:
weights = pd.DataFrame(weight_ih_np, columns=features)

In [55]:
weights['rWrist_x_minus_80']

0     -0.118293
1      0.057691
2     -0.099260
3      0.020036
4      0.046773
         ...   
97    -0.108127
98    -0.147265
99     0.120991
100   -0.109019
101    0.127330
Name: rWrist_x_minus_80, Length: 102, dtype: float32

In [56]:
weights['lWrist_x_minus_80']

0      0.073618
1      0.129745
2     -0.052349
3      0.136418
4      0.078713
         ...   
97    -0.137238
98    -0.041288
99    -0.127240
100   -0.085322
101   -0.029799
Name: lWrist_x_minus_80, Length: 102, dtype: float32

In [121]:
# Hyperparameters
input_dim = 1
hidden_dim = 1
output_dim = 4
num_layers = 1
n_epochs =151
lr = 0.01

# Create an instance of GRUCellNet
model = GRUCellNet(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function and optimizer
class_weights = torch.tensor([1.8]).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)  
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [122]:
rwrist_df = scaled_df[['rWrist_x','trial','id','s_1','s_2','s_3','s_4']]
lwrist_df = scaled_df[['lWrist_x','trial','id','s_1','s_2','s_3','s_4']]
labels = ['s_1','s_2','s_3','s_4']

In [48]:
y_train.to_numpy()

array([[1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       ...,
       [1., 1., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 0., 0.]])

In [36]:
output.shape

torch.Size([5328, 4])

In [99]:
rWrist_loss.requires_grad_(True)

tensor(0.8470, dtype=torch.float64, requires_grad=True)

In [29]:
load_folder = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models'
model_filename = 's_m80_val13_gates.pth'

# Load the model
checkpoint = torch.load(os.path.join(load_folder, model_filename))
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [49]:
#filtered_df = scaled_df[scaled_df['trial'] != 14]
#target = filtered_df[['s_1','s_2','s_3','s_4']] 
target = torch.from_numpy(y_train.to_numpy())
target.shape

torch.Size([5328, 4])

In [87]:
input.grad

AttributeError: 'function' object has no attribute 'grad'

In [51]:
model_loss = loss_fn(output, target)
model_loss.requires_grad_(True)
model_loss.backward()


# Access gradients for specific parameters
gru_weight_ih_gradients = model.gru.weight_ih.grad  # Gradients for input-hidden weights of GRU
gru_weight_hh_gradients = model.gru.weight_hh.grad  # Gradients for hidden-hidden weights of GRU
linear_weight_gradients = model.fc.weight.grad  # Gradients for the weight of the linear layer

In [80]:
model_loss

tensor(0.6458, dtype=torch.float64, requires_grad=True)

In [79]:
gradients_array = gru_weight_ih_gradients.numpy()
gradients = pd.DataFrame(gradients_array, columns=features)
gradients[['rWrist_x_minus_80','lWrist_x_minus_80']]



,rWrist_x_minus_80,lWrist_x_minus_80
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
97,0.0,0.0
98,0.0,0.0
99,0.0,0.0
100,0.0,0.0


In [76]:
np.round(np.mean(np.round(gradients['rWrist_x_minus_80'] - gradients['lWrist_x_minus_80'],5)),15)

5.392157618e-06

In [106]:
# Perform the backward pass on the combined loss
combined_loss = rWrist_loss + lWrist_loss
combined_loss.backward()
gradients = combined_loss.grad
print(gradients)

None


c:\Users\kacpe\anaconda3\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\build\aten\src\ATen/core/TensorBody.h:482.)
  return self._grad


In [103]:
lWrist_gradients

tensor(1., dtype=torch.float64)

In [ ]:
        # Iterate through the test data batches
        for inputs, _ in test_dataloader:
            inputs = inputs.float()
            # Forward pass to get predictions
            with torch.no_grad():
                predictions, h, test_hidden_states = model(inputs)
                probabilities = torch.sigmoid(predictions)
                preds = torch.round(probabilities)

In [65]:
        # Define the folder and model filename
load_folder = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models'
model_filename = 'lWrist_model.pth'

# Load the model
checkpoint = torch.load(os.path.join(load_folder, model_filename))
model.load_state_dict(checkpoint)

# Set the model in evaluation mode
model.eval()
# Set the model in evaluation mode


# Access the model's state dictionary
model_state_dict = model.state_dict()

# Now you can access the weights of the nn.GRUCell layers
gru_cell_weights = {
    key: value
    for key, value in model_state_dict.items()
    if key.startswith('gru_cells.')
}

    


# Extract and assign the weights to separate variables
lWrist_weights_ih = gru_cell_weights['gru_cells.0.weight_ih']
lWrist_weights_hh = gru_cell_weights['gru_cells.0.weight_hh']


        # Define the folder and model filename
load_folder = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models'
model_filename = 'rWrist_model.pth'

# Load the model
checkpoint = torch.load(os.path.join(load_folder, model_filename))
model.load_state_dict(checkpoint)

# Set the model in evaluation mode
model.eval()
# Set the model in evaluation mode


# Access the model's state dictionary
model_state_dict = model.state_dict()

# Now you can access the weights of the nn.GRUCell layers
gru_cell_weights = {
    key: value
    for key, value in model_state_dict.items()
    if key.startswith('gru_cells.')
}

    


# Extract and assign the weights to separate variables
rWrist_weights_ih = gru_cell_weights['gru_cells.0.weight_ih']
rWrist_weights_hh = gru_cell_weights['gru_cells.0.weight_hh']

RuntimeError: Error(s) in loading state_dict for GRUCellNet:
	size mismatch for gru.weight_ih: copying a param with shape torch.Size([3, 1]) from checkpoint, the shape in current model is torch.Size([102, 46]).
	size mismatch for gru.weight_hh: copying a param with shape torch.Size([3, 1]) from checkpoint, the shape in current model is torch.Size([102, 34]).
	size mismatch for gru.bias_ih: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([102]).
	size mismatch for gru.bias_hh: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([102]).
	size mismatch for fc.weight: copying a param with shape torch.Size([4, 1]) from checkpoint, the shape in current model is torch.Size([4, 34]).
	size mismatch for gru_cells.0.weight_ih: copying a param with shape torch.Size([3, 1]) from checkpoint, the shape in current model is torch.Size([102, 46]).
	size mismatch for gru_cells.0.weight_hh: copying a par

In [119]:
print(lWrist_weights_ih)
print(rWrist_weights_ih)

tensor([[ 0.6961],
        [ 0.6693],
        [-0.4421]])
tensor([[ 0.6961],
        [ 0.6693],
        [-0.4421]])


In [78]:
rwrist_df = scaled_df[['rWrist_x','trial','id','s_1','s_2','s_3','s_4']]
lwrist_df = scaled_df[['lWrist_x','trial','id','s_1','s_2','s_3','s_4']]
labels = ['s_1','s_2','s_3','s_4']

#rwrist_df = lwrist_df.copy()
set_values = 13 
train_set = rwrist_df[rwrist_df['trial']!=set_values].drop(columns=['id', 'trial'])
test_set = rwrist_df[rwrist_df['trial']==set_values].drop(columns=['id', 'trial'])
full_set = rwrist_df.drop(columns=['id','trial'])

# split data into x and y 
X_train, y_train = train_set.drop(columns=labels), train_set[labels]
X_test, y_test = test_set.drop(columns=labels), test_set[labels]
X, y = full_set.drop(columns=labels), full_set[labels]
print(X_train.columns)
# reset index 
X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)
X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
X, y = X.reset_index(drop=True), y.reset_index(drop=True) 
print(X_train.columns)
# Create custom datasets for training, validation, and testing
full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)
#print(X_train.shape[0])


train_losses, train_accs, train_predicted_labels, train_probs_final, train_hidden_states = run_training(
    train_dataloader, val_dataloader=None, model=model, optimizer=optimizer, loss_fn=loss_fn, num_epochs=n_epochs, scheduler=scheduler)


state_dict = model.state_dict()
folder_path = 'C:/Users/kacpe/Desktop/study/research lab/lab_rotation_git/gradient_models' 
model_filename = 'rWrist_model.pth'  

# Combine the folder path and model filename
full_model = os.path.join(folder_path, model_filename)

        # Save the model to the specified folder
torch.save(state_dict, full_model)

# Iterate through the test data batches
for inputs, _ in train_dataloader:
    inputs = inputs.float()
    inputs.requires_grad = True  # Set requires_grad to True
    # Forward pass to get predictions
    with torch.no_grad():
        predictions, h, test_hidden_states = model(inputs)
        probabilities = torch.sigmoid(predictions)
        rWrist_output = torch.round(probabilities)

Index(['rWrist_x'], dtype='object')
Index(['rWrist_x'], dtype='object')


RuntimeError: input has inconsistent input_size: got 1 expected 46